##### Flow:
1. load, reshape and split in test and train all the images
2. instantiate a cnn model 
3. run the model on train and test data to verify how many epochs are more or less needed to get a nice model
4. use that number of epochs to run cross validation (pass the whole 'set_' of images to the cross validation). Repeat from 2 with another model and compare.


- If you notice that you hardly overfit maybe remove/decrease the dropout layers (e.g. from 0.25 to 0.15)
- Try building a model that predicts directly 16x16

In [53]:
%matplotlib inline

import matplotlib.pyplot as plt
from preprocessing import *
from cnn_models import *
from datetime import datetime
from evaluate import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Overview
The goal here is to use the CNN to reduce the size of the input image to obtain a "discretized" image of shape, e.g. (W/16, H/16). Every entry of this image is related to a patch in the input image. This obtained image is compared by the CNN with the groundtruth (after properly discretizing by it patch-wise).

### - Load data

In [3]:
# Loaded a set of images
n = 2

imgs, gt_imgs = load_images(n)
imgs[0].shape, gt_imgs[0].shape

Loading 2 images


((400, 400, 3), (400, 400))

### - Reshape the data
We reshape each input to fulfill our cnn inputs and output shape.

In [4]:
# !!! set predict_patch_width in accordance to the model you are using !!!
# the shape of the output of the model depends on the strides parameters 
# (if a layer has stride=2 then each ouput's side is half of the input'side).
# predict_patch_width must be equal to the total reduction of the model, e.g.
# if the model has three layer with stride=2 => the input of the model is 
# reduced by a factor of 2*2*2=8, i.e. the ouptut will be patch-wise with 
# patches 8x8 pixels.
predict_patch_width = 8

X, Y = images_to_XY(imgs, gt_imgs, predict_patch_width=predict_patch_width)

set_ = SimpleNamespace()
set_.X = X
set_.Y = Y

X.shape, Y.shape

((2, 400, 400, 3), (2, 50, 50))

### - For now avoid cross validation, just split the datasest in test and train. 

In [ ]:
test_ratio = 0.25

train, test = split_train_test(X, Y, test_ratio=test_ratio, seed=1)
train.X.shape, train.Y.shape, test.X.shape, test.Y.shape 

In [ ]:
# # check it makes sense (show the i-th input of set_)
# i = 0
# set_ = test

# fig, axs = plt.subplots(1, 2, figsize=(20, 10))
# axs[0].imshow(set_.Y[i, :, :, 1], cmap='gray')
# axs[1].imshow(set_.X[i, :, :])

### - Build the CNN model or load a previous one

- Choose one of the models you defined (with model_n) and initialize it.

In [ ]:
# generate an unique name for the model (so to avoid overwriting previous models)
folder_name = "model_"+str('{0:%Y-%m-%d_%H:%M:%S}'.format(datetime.now()))
model_path = "models/"+folder_name
model = CnnModel(model_n=0, model_path=model_path)
model.summary()

- Otherwise load a previous model

In [ ]:
# give the folder
folder_name = "model_2017-12-14_194132"
model_path = "../models/"+folder_name
model = CnnModel(model_path=model_path)
model.load() # load the model and its weights
model.summary()

### - Train the model on the train data while validating it on the test data

In [ ]:
# pass a batch size which is a factor of train.shape[0] so that all the batches are fo the same size
num_epochs=1
batch_size=3
_ = model.train(train, test=test, num_epochs=num_epochs, batch_size=batch_size, monitor='val_loss') 

### - Run cross validation to evaluate the model

In [ ]:
result = model.cross_validation(set_, batch_size=batch_size, num_epochs=num_epochs)
result

In [ ]:
# plot the histories of the cross validation
plot_history(result["history_mean"]) 
# history of the folds (check if there is a worst case)
# plot_history(result["histories"][0]) 
# plot_history(result["histories"][1]) 
# plot_history(result["histories"][2]) 
# plot_history(result["histories"][3]) 

#### Plot the accuracy and the loss obtained during training

In [ ]:
last_epochs=1000 # plot only the last n epochs
model.plot_history(last_epochs=last_epochs)

#### Display the output of a specific layer

In [ ]:
# these are all the layers 
model.model.layers

In [ ]:
# choose a layer and an image 
image = set_.X[0]
layer_num = 1

model.show_layer_output(image, layer_num, filename="") # pass a filename if you want to store the image to file 

### - Evaluate the model on the test data

In [ ]:
# check the performance on train or test
set_ = set_

model.evaluate_model(set_.X, set_.Y)

### - Show a prediction

In [ ]:
# choose an image to predict (or part of it)
img = test.X[0][:, :]

model.display_prediction(img, ax=None)

### - Save/load model

In [ ]:
model.save()

## - Postprocessing

#### 1. Predict an image after rotating and flipping in some predefined ways and then average the predictions. 

- **predict**

In [ ]:
set_.X.shape, set_.Y.shape

Normal prediction

In [ ]:
pred_norm = model.predict(set_.X)
pred_norm.shape

Predict augmenting the image: 4 rotations

In [ ]:
pred_rot4 = model.predict_augmented(set_.X, n_rotations=4)
pred_rot4.shape

Predict augmenting the image: 8 rotations

In [ ]:
pred_rot8 = model.predict_augmented(set_.X, n_rotations=8)
pred_rot8.shape

- ** evaluate **

Evaluate both and compare the F1 scores

In [36]:
true = set_.Y.flatten()
pred_norm_class = predictions_to_class(pred_norm).flatten()
pred_rot4_class = predictions_to_class(pred_rot4).flatten()
pred_rot8_class = predictions_to_class(pred_rot8).flatten()
pred_norm_class.shape, pred_rot4_class.shape, pred_rot8_class.shape, true.shape

((5000,), (5000,), (5000,), (5000,))

In [ ]:
evaluate_predictions(pred_norm_class, true)

In [ ]:
evaluate_predictions(pred_rot4_class, true)

In [ ]:
evaluate_predictions(pred_rot8_class, true)

In [ ]:
# im_rot = rotate(im, 45, reshape=True, order=1, mode="reflect")
# plt.figure(figsize=(10, 10))
# plt.imshow(im_rot)
# plt.show()

# pred = model.predict(np.array([im_rot]))[0]
# pred = take_image_at_center(rotate_image(pred, -45), target_shape=(50, 50))
# plt.figure(figsize=(10, 10))
# plt.imshow(prediction_to_class(pred), cmap="gray")
# plt.show()

- Choose the best threshold to classify a patch as road.

In [54]:
grid_search_treshold(pred_norm[:, :, :, 1].flatten(), true)

(0.91435875585854265, 0.65000000000000002)

In [55]:
grid_search_treshold(pred_rot4[:, :, :, 1].flatten(), true)

(0.941424357353561, 0.65000000000000002)

In [56]:
grid_search_treshold(pred_rot8[:, :, :, 1].flatten(), true)

(0.92609228256431197, 0.55000000000000004)

### - Others

In [ ]:
# some callbacks example: 

# create a list of callbacks we want to use during training
# # a callback to store epoch results to a csv file
# filename='model_train_new.csv'
# csv_log = callbacks.CSVLogger(filename, separator=',', append=False)

# # a callback to stob before doing the predefined number of epochs (stop before overfitting the data)
# early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='min')

# # a callback to save the best model (best model = the one with the lowest 'monitor' variable)
# filepath = "best-weights-{epoch:03d}-{loss:.4f}-{acc:.4f}.hdf5"
# checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# # callbacks_list = [csv_log,early_stopping,checkpoint]

print(
    "-get configurations:", "\n",
    model.get_config(), "\n",
    model.layers[0].get_config(), "\n",

    "\n-get shapes", "\n",
    model.layers[0].input_shape, "\n",
    model.layers[0].output_shape, "\n",
    
    "\n-get weights", "\n",
    model.layers[0].get_weights()[0].shape, "\n",
    
    "\n-check if trainable", "\n",
    model.layers[0].trainable, "\n", # you can set this to false to "freeze" a layer
)

In [ ]:
from IPython.core.debugger import Pdb
debugger = Pdb()
debugger.set_trace() # put this line as a breakpoint

Test batch generation

In [ ]:
gen1, gen2 = image_generators(X, Y) 

In [ ]:
from cnn_models import batches_generator
j = 0
x_batches = []
y_batches = []

for x, y in batches_generator(X[:4], Y[:4], batch_size = 4):
    j += 1
    if j > 10:
        break
    x_batches.append(x)
    y_batches.append(y)

In [ ]:
np.array(x_batches).shape, np.array(y_batches).shape

In [ ]:
b = 0
i = -1

In [ ]:
i += 1
if i >= x_batches[0].shape[0]:
    i = 0
    b += 1
print("Batch", str(b) + ". Image", i)
fig, axs = plt.subplots(1, 2)
fig.set_size_inches((20, 10))
axs[0].imshow(x_batches[b][i], cmap='gray')
axs[1].imshow(y_batches[b][i][:, :, 1], cmap='gray')